# Agregación y agrupación

Una parte esencial del análisis de grandes cantidades de datos es la sumarización eficiente; la capacidad de hacer operaciones tales como ``sum()``, ``mean()``, ``median()``, ``min()``, and ``max()`` dónde un sólo número da visión de la naturaleza de una gran cantidad de datos. En la clase de hoy vamos a explorar las agregaciones que nos ofrece Pandas, desde las más simple, que ya hemos visto y trabajado con los Numpy Arrays, a los más sofisticados basados en el concepto de ``GroupBy``.

Cómo hemos usado en otros capítulos, vamos a usar la función ``display()``:

In [ ]:
import numpy as np
import pandas as pd

class display(object):
    """Representador HTML de múltiples objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## Planets Dataset

Para explicar las bases, vamos a usar el **Dataset** de Planets, disponible con el paquete de ``Seaborn``, que ya descubriremos en el módulo de visualización. Da información de los planetas que los astrónomos han descubierto orbitando en otras estrellas (conocidos como *planetas extrasolares* o *exoplanetas*). Puede ser descargado con un comando de la librería ``Seaborn``

In [ ]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

In [ ]:
#vemos como es el data set- info() head() tail() describe()

This has some details on the 1,000+ extrasolar planets discovered up to 2014.

## Agregación básica en Pandas

Anteriormente, ya hemos visto algunas de las funciones de agregación que teníamos en Numpy. Con un sólo nivel dimensional, la agregación funciona así para una ``Series``:

In [ ]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

In [ ]:
ser.sum()

In [ ]:
ser.mean()

Para un ``Dataframe``, la agregación devuelve un resultado **para cada columna**:

In [ ]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

In [ ]:
df.mean(axis=0)

Si así lo disponemos con el argumento ``axis``, podemos agregar **en cada fila**:

In [ ]:
# si quiero ver la media por filas 
df.mean(axis='columns')

#df.mean(axis=1) si lo preferimos 

Las ``Series`` y ``Dataframe`` de Pandas incluye todos los tipos de agregaciones que hemos visto ya para los Numpy Arrays, pero además, tenemos el método ``describe()`` que computa distintos agregaciones estándar para cada columna para darnos información clave:

In [ ]:
planets.info()

In [ ]:
# Elimina todas las filas que contengan valores nulos (NaN)
# y luego muestra un resumen estadístico de las columnas
planets.dropna().describe()

Esto es muy útil para comenzar a entender de manera general las propiedades del Dataset. Por ejemplo, sabemos por la columna ``year`` que el primer exoplaneta fue descubierto en 1989, y que la mitad de ellos no habían sido descubiertos antes del año 2009. Esto es gracias a la misión *Kepler*, que es un telescopio espacial que está específicamente para buscar planetas que eclipsan a otras estrellas. 

Las siguientes agregaciones vienen con el paquete de Pandas:

| Agregación              | Descripción                     |
|--------------------------|---------------------------------|
| ``count()``              | Número total de elementos          |
| ``first()``, ``last()``  | Primer y último elemento             |
| ``mean()``, ``median()`` | Media y mediana                 |
| ``min()``, ``max()``     | Mínimo y máximo             |
| ``std()``, ``var()``     | Desviación estándar y varianza |
| ``mad()``                | Desviación media absoluta         |
| ``prod()``               | Producto de todos los elementos            |
| ``sum()``                | Suma de todos los elementos               |

Todos están presentes como objetos de ``Dataframe`` y ``Series``.

Para ir más allá de los datos estas agregaciones no son suficiente. El siguiente nivel de sumarización es el conocido ``groupby``, que nos permite procesar subsets de datos de manera rápida y eficiente.

## GroupBy: Split, Apply, Combine

Agregaciones más simples nos permiten saborear el dataset, pero casi siempre preferiremos agregar condicionalmente en algún o algunas dimensiones/índices: esto se implementa con la operación ``groupby``.

El nombre de *Group By* viene de un comando de **SQL**, pero quizás es más explicativo verlo por el término que describió Hadley Wickham de RStats: *split, apply, combine*.

### Split, apply, combine

Un ejemplo muy canónico de este término *split-apply-combine* es el de agregar en forma de suma.

Esto nos ayuda a aclarar lo que ``groupby`` realiza:

- El paso de **split/separar** involucra romper y agrupar el ``Dataframe`` dependiento del valor de una clave especificada.
- El paso de **apply/aplicar** involucra computar alguna función, usualmente una agregación, una transformación, un filtrado entre esos grupos individuales.
- El paso de **combine/combinar** une esos resultados en un array de salida/output

Mientras esto podría ser realizado de manera manual usando una combinación de *masking*, agregación y unión que ya hemos visto antes, existe un pero importante, **que las agregaciones a realizar no tienen porqué ser instanciadas**. En lugar de eso, ``groupby`` puede (casi todas las veces) hacer esto en una sola llamada a los datos, realizando automáticamente el cálculo de la agregación para cada grupo de una sola vez. El poder de ``Groupby`` radica en hacer esos pasos de manera combinada por nosotros: El usuario no necesita pensar en cómo va a hacer la computación.

Como ejemplo, vamos a usar Pandas para la computación en base al siguiente diagrama:

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({'department': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'VV': range(6)})
df

La versión más simple de *split-apply-combine* puede ser realizado con el método ``groupby()``, pasando el nombre de la key a agregar como argumento:

In [ ]:
df.groupby('department')

Date cuenta que lo que ha devuelto no es un ``Dataframe``, es un objeto de ``DataFrameGroupBy``. Este objeto en dónde la mágia ocurre, puedes pensar en el cómo una vista especial de un ``Dataframe``, en dónde tiene la instrucción de cómo se van a distribuir los grupos pero no va a realizarse hasta que la agregación sea **aplicada**. Esta evaluación difusa o *"lazy evaluation"* significa que agregaciones comunes como las que hemos presentado podrían ser implementadas facilmente y de manera transparente para el usuario.

Para producir un resultado, **debemos agregar este objeto**, cosa que nos dará un resultado en base a la agregación:

In [ ]:
df['VV'].mean()

In [ ]:
# Agrupa el DataFrame por la columna 'department'
# y calcula la media de todas las columnas numéricas en cada grupo
df_grouped = df.groupby('department').mean()



El método de ``sum()`` es solo una de las posibilidades aquí, podemos aplicar culqueir tipo de función de agregación de Pandas o de Numpy, además, podemos aplicar de manera simultánea cualquier operación al ``Dataframe``. Ahora lo veremos en detalle.

### El objeto Groupby

El objeto ``Groupby`` es una abstracción muy flexible, se podría tratar como una colección de un ``Dataframe``. Vamos a ver a continuación ejemplos con nuestro ``Dataframe`` de Planets:

#### Indexado de columnas

Ya lo adelantabamos anteriormente, el objeto ``Groupby`` soporta la indexación de la misma manera que el ``Dataframe``, devolviendo una modificación del objeto ``Groupby``:

In [ ]:
planets.head()

In [ ]:
planets['method'].unique()

In [ ]:
planets.groupby('method')

In [ ]:
# Agrupa el DataFrame 'planets' por la columna 'method'
# (el método con el que se descubrió el planeta)
# y calcula la media de la columna 'orbital_period' para cada grupo
planets.groupby('method')['orbital_period'].mean()

Aquí hemos seleccionado un grupo concreto de tipo Series del DataFrame original haciendo referencia a su nombre de columna.
Al igual que con el objeto GroupBy, no se realiza ningún cálculo hasta que llamamos a alguna función de agregación sobre el objeto.

In [ ]:
planets.head()

In [ ]:
# Extrae la columna 'method' del DataFrame planets
# Obtiene los valores únicos en esa columna con .unique()
# Cuenta cuántos métodos distintos hay con len()

len(planets['method'].unique())

In [ ]:
planets.groupby('method')[['orbital_period']].mean()

In [ ]:
# media del período orbital de los planetas descubiertos por Astrometry (codigo explicado en la siguiente celda)
planets[planets['method']=='Astrometry'][['orbital_period']].mean()

In [ ]:
#El codigo anterior paso a paso 

# Filtra el DataFrame para quedarte solo con las filas donde
# la columna 'method' sea exactamente 'Astrometry'
planets[planets['method'] == 'Astrometry']

# De ese subconjunto selecciona la columna 'orbital_period'
planets[planets['method'] == 'Astrometry'][['orbital_period']]

# Calcula la media de los períodos orbitales en ese subconjunto
planets[planets['method'] == 'Astrometry'][['orbital_period']].mean()

Nos hacemos una idea de la escala general de los periodos orbitales en días que cada método de observación es capaz de abarcar.

#### Iteración entre grupos

El objeto ``Groupby`` soporta la iteración directa entre grupos, devolviendo cada grupo como una ``Series`` o un ``Dataframe``:

In [ ]:


# Recorremos el DataFrame agrupado por la columna 'method'
# .groupby('method') agrupa las filas que tienen el mismo valor en esa columna.
# Cada grupo será un subconjunto del DataFrame original.
for (method, group) in planets.groupby('method'):
    
    # En cada iteración:
    #  - 'method' → es el nombre del grupo (por ejemplo: 'Transit', 'Radial Velocity', etc.)
    #  - 'group' → es un nuevo DataFrame que contiene solo las filas de ese método
    
    # .shape devuelve una tupla (filas, columnas)
    # En este contexto indica cuántas observaciones hay en ese grupo
    # y cuántas columnas tiene (por ejemplo: (397, 6) → 397 filas, 6 columnas)
    
    # .format() se usa para imprimir texto con formato.
    # {0:30s} → reserva 30 caracteres para el primer valor (cadena) alineado a la izquierda.
    # {1} → inserta el segundo valor (aquí la tupla del tamaño del grupo)
    
    print("{0:30s} shape={1}".format(method, group.shape))

# ============================================
# EJEMPLO DE SALIDA:
# Astrometry                     shape=(2, 6)
# Eclipse Timing Variations      shape=(9, 6)
# Imaging                        shape=(38, 6)
# Microlensing                   shape=(23, 6)
# Orbital Brightness Modulation  shape=(3, 6)
# Pulsar Timing                  shape=(5, 6)
# Pulsation Timing Variations    shape=(1, 6)
# Radial Velocity                shape=(553, 6)
# Transit                        shape=(397, 6)
# Transit Timing Variations      shape=(4, 6)
# ============================================

# INTERPRETACIÓN:
# Cada línea muestra un método de descubrimiento de planetas
# (columna 'method') y cuántas veces aparece en el DataFrame.
# Ejemplo:
#   - 'Radial Velocity' tiene 553 registros (553 planetas detectados por ese método)
#   - 'Transit' tiene 397 registros
#   - 'Astrometry' solo tiene 2 registros

# ============================================
# OPCIONAL (versión alternativa más compacta):
# Si solo quieres ver el conteo de filas por grupo sin recorrer con for:
# planets.groupby('method').size()
# Esto devuelve una Serie con el número de filas por método.
# ============================================


Esto puede ser útil para hacer ciertas operaciones de manera más manual, aunque es más rápido usar la funcionalidad de ``apply``, que veremos a continuación.

#### Métodos de envío

A través de la magia de las clases de Python, cualquier método no implementado explícitamente por el objeto ``GroupBy`` será pasado y llamado en los grupos, ya sean objetos ``DataFrame`` o ``Series``.
Por ejemplo, puedes utilizar el método ``describe()`` de ``DataFrame`` para realizar un conjunto de agregaciones que describan cada grupo en los datos:

In [ ]:
# agrupa por 'method' y genera estadísticas descriptivas de la columna 'year'
temp = planets.groupby('method')['year'].describe()  
temp  


Mirar esta tabla nos ayuda a entender mejor los datos: por ejemplo, la gran mayoría de los planetas se han descubierto por los métodos de *Radial Velocity* y *Transit*, aunque este último sólo se hizo común (debido a los nuevos y más precisos telescopios) en la última década.
Los métodos más recientes parecen ser el de la *Transit Timing Variations* y el de la *Orbital Brightness Modulation*, que no se utilizaron para descubrir un nuevo planeta hasta 2011.

Este es sólo un ejemplo de la utilidad de los métodos de envío. Fíjate en que se aplican *a cada grupo individual*, y los resultados se combinan dentro de ``GroupBy`` y se devuelven. De nuevo, cualquier método válido de ``DataFrame`` o ``Series`` puede utilizarse en el objeto ``GroupBy`` correspondiente, ¡lo que permite realizar operaciones muy flexibles y potentes!

### Aggregate, filter, transform y apply

Antes nos hemos centrado en la agregación para la operación de combinación, pero hay más opciones disponibles. En particular, los objetos ``GroupBy`` tienen los métodos ``agregate()``, ``filter()``, ``transform()``, y ``apply()`` que implementan eficientemente una variedad de operaciones útiles antes de combinar los datos agrupados.

En las siguientes subsecciones, utilizaremos este ``DataFrame``:

In [ ]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'department': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'anio': [2020,2020,2020,2021,2021,2021],
                   'VV': rng.randint(0, 10, 6)},
                   columns = ['department', 'anio', 'VV'])
df

#### Agregación

Ya estamos familiarizados con las agregaciones ``GroupBy`` con ``sum()``, ``median()``, y similares, pero el método ``aggregate()`` permite una flexibilidad aún mayor.
Puede tomar una cadena, una función, o una lista de ellas, y calcular todos los agregados a la vez.

Aquí hay un ejemplo rápido que combina todo esto:

In [ ]:
df.groupby('department').median()

In [ ]:
# Agrupar el DataFrame por la columna 'department'
# .aggregate() permite aplicar una o varias funciones de agregación a cada grupo
# En este caso: calcula el mínimo (min), la mediana (np.median) y el máximo (max) por grupo
df_grouped = df.groupby('department').aggregate(['min', np.median, max])

df_grouped

Otro patrón útil es pasar un diccionario que asigna los nombres de las columnas a las operaciones que deben aplicarse a esa columna:

In [ ]:
df

In [ ]:
# Agrupar el DataFrame por la columna 'department'
# Usamos .aggregate() con un diccionario para aplicar funciones específicas a columnas concretas:
#   - En la columna 'anio' se calcula el valor mínimo
#   - En la columna 'VV' se calcula el valor medio (promedio) 

df_grouped = df.groupby('department').aggregate({'anio': 'min',
                                                'VV': 'mean'}).rename(columns={'anio':'anio_min', # Renombramos la columna resultante 'anio' -> 'anio_min'
                                                                                "VV":"VV_mean"}) # Renombramos la columna resultante 'VV' -> 'VV_mean' 
df_grouped

In [ ]:
# Agrupar el DataFrame por las columnas 'anio' y 'department'
# Para cada combinación (anio, department) calcula la media de todas las columnas numéricas
# A diferencia del caso anterior, aquí no se resume todo el departamento en una fila,
# sino que se obtiene el detalle año por año para cada departamento
# (si se quisiera que 'anio' y 'department' no fueran índice, se podría usar as_index=False)
df_grouped = df.groupby(["anio", "department"]).mean() 

df_grouped

#### Filtrado

Una operación de filtrado permite descartar datos en función de las propiedades del grupo.
Por ejemplo, podríamos querer mantener todos los grupos en los que la desviación estándar es mayor que algún valor crítico:

In [ ]:
#Creamos una funcion de filtrado que usaremos luego
    
def filter_func(x):
    return x['VV'].min() > 1

In [ ]:
#Recordamos la sintaxis mascara boleana 

df['VV'] > 0

In [ ]:
# Recordamos la sintaxis mascara boleana aplicada

df[df['VV'] > 0]

In [ ]:
# ¿La función display del principio del notebook sirve para esto?
# Sí. Asegúrate de importarla desde IPython.display.
from IPython.display import display, Markdown

# Mostrar el DataFrame original 'df'
# Luego el resultado de agrupar por 'department' y calcular el mínimo de cada grupo
# Finalmente el DataFrame filtrado con filter_func (min(VV) > 1)

min_by_dept = df.groupby('department').min()
filtered = df.groupby('department').filter(filter_func)

display(Markdown("**DataFrame original (`df`)**"))
display(df)

display(Markdown("**Mínimos por `department`**"))
display(min_by_dept)

display(Markdown("**Filtrado con `filter_func` (min(VV) > 1)**"))
display(filtered)

# Si prefieres todo en una sola llamada (sin títulos), también funciona:
# display(df, min_by_dept, filtered)


La función de filtrado debe devolver un valor booleano que especifica si el grupo pasa el filtrado. No pasa el grupo B al no superar uno de sus registros el hecho de ser superior de cero.

#### Transformación

Mientras que la agregación debe devolver una versión reducida de los datos, la transformación puede devolver alguna versión transformada de los datos completos para recombinar.
Para tal transformación, la salida tiene la misma forma que la entrada.
Un ejemplo común es centrar los datos restando la media del grupo:

In [ ]:
# Diferencia entre .aggregate() y .transform():
# - .aggregate() (o .agg()) devuelve un resultado reducido (por ejemplo: min, max, mean), 
#   es decir, colapsa el grupo en un único valor por función.
# - .transform(), en cambio, devuelve un resultado del mismo tamaño que el grupo original.
#   Esto permite "transformar" cada valor y luego recombinar en un DataFrame de la misma forma que el original.

# Ejemplo común de transformación: centrar los datos de cada grupo restando la media del grupo
# La salida tiene la MISMA forma que la entrada, pero transformada.

# Definimos una función personalizada que normaliza: resta la media y divide por la desviación estándar del grupo
def mi_funcion(x):
    return (x - x.mean()) / x.std()

# Aplicamos la función a cada grupo con transform: normalización dentro de cada 'department'
df_normalizado = df.groupby('department').transform(mi_funcion)

# Otro ejemplo: solo centrar (restar la media del grupo)
df_centrado = df.groupby('department').transform(lambda x: x - x.mean())

# Mostrar resultados
display(
    df.head(),             # Datos originales (primeras filas)
    df_normalizado.head(), # Datos normalizados (media=0, std=1 dentro de cada departamento) 
    df_centrado.head()     # Datos centrados (media=0 dentro de cada departamento)
)


[Anexo explicacion resultados](Anexo_transformacion.ipynb)


----


### Mini bonus 
[Que es Lambda](https://ellibrodepython.com/lambda-python)

------

#### El método apply()

El método ``apply()`` permite aplicar una función arbitraria a los resultados del grupo.
La función debe procesar un ``DataFrame``, y devolver un objeto Pandas (por ejemplo, ``DataFrame``, ``Series``) o un escalar; la operación de combinación se adaptará al tipo de resultado devuelto.

Por ejemplo, aquí hay un ``apply()`` que normaliza la primera columna por la suma de la segunda:

In [ ]:
# abro aply para visualizar (no hacerle mucho caso)

import numpy as np
import pandas as pd

class display(object):
    """Representador HTML de múltiples objetos"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

In [ ]:
# Definimos una función personalizada que recibe un grupo (sub-DataFrame)
def norm_by_data2(x):
    # Dentro de cada grupo, dividimos la columna 'anio'
    # entre la suma de la columna 'VV' del mismo grupo
    # Esto normaliza los valores de 'anio' usando la "escala" de 'VV'
    x['anio'] /= x['VV'].sum()
    return x  # devolvemos el grupo transformado

# Mostrar el DataFrame original y luego el resultado de aplicar la función por grupo
# 'department' define los grupos
display('df', "df.groupby('department').apply(norm_by_data2)")

``apply()`` dentro de un ``GroupBy`` es bastante flexible: el único criterio es que la función toma un ``DataFrame`` y devuelve un objeto Pandas o un escalar; ¡lo que hagas en medio depende de ti!

### Especificar una key de separación

En los ejemplos simples presentados anteriormente, agrupamos el ``DataFrame`` en un solo nombre de columna.
Esta es sólo una de las muchas opciones por las que los grupos pueden ser definidos, y vamos a ir a través de algunas otras opciones para la especificación de los grupos.

#### Una lista, array, series, o index dando los grupos de antemano

La clave puede ser cualquier serie o lista cuya longitud coincida con la del ``DataFrame``. Por ejemplo:

In [ ]:
# Creamos una lista L con etiquetas numéricas que servirá para agrupar las filas
L = [0, 1, 0, 1, 2, 0]

# Usamos groupby(L) para agrupar el DataFrame 'df' siguiendo las etiquetas de la lista
# Luego aplicamos .sum() para sumar las columnas numéricas dentro de cada grupo
# Finalmente, mostramos el DataFrame original y el resultado agrupado
display('df', 'df.groupby(L).sum()')


Por supuesto, esto significa que hay otra forma más explicita de realizar el ``df.groupby('key')`` de antes:

In [ ]:
display('df', "df.groupby(df['department']).sum()")

#### Un índice mapeado de un diccionario o una serie a un grupo

Otro método es proporcionar un diccionario que asigne los valores del índice a las claves del grupo:

In [ ]:
# Cambiar el índice del DataFrame 'df' a la columna 'department'
df2 = df.set_index('department')

# Creamos un diccionario de mapeo: cada clave es un valor del índice 'department'
# y cada valor indica a qué grupo queremos asignar ese departamento
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}

# Agrupar 'df2' según el diccionario de mapeo aplicado al índice
# Luego sumar las columnas numéricas dentro de cada grupo ('vowel', 'consonant')
# Finalmente, mostrar el DataFrame transformado
display('df2', 'df2.groupby(mapping).sum()')


#### Cualquier función de python

Al igual que el *mapping*, puedes pasar cualquier función de Python que introduzca el valor del índice y del grupo resultado:

In [ ]:
# Mostrar el DataFrame 'df2'
# Luego agrupar usando groupby(str.lower):
#   - str.lower se aplica sobre el índice de 'df2' (que en este caso son los departamentos)
#   - Convierte cada etiqueta del índice a minúsculas
#   - Así, si hubiera diferencias de mayúsculas/minúsculas ('A' y 'a'), se agrupan juntas
# Después calculamos la media de cada grupo con .mean()
display('df2', 'df2.groupby(str.lower).mean()')

#### Una lista de *keys* válidas

Además, cualquiera de las opciones de clave anteriores puede combinarse para agruparse en un índice múltiple:

In [ ]:
# Agrupar el DataFrame df2 usando una lista de funciones/estructuras:
# 1. str.lower → se aplica sobre el índice (department), convirtiéndolo a minúsculas
# 2. mapping   → es un diccionario que asigna cada departamento a una categoría ('vowel' o 'consonant')
#
# El resultado es un groupby con un índice jerárquico (MultiIndex):
#   - Primer nivel: el nombre del departamento en minúsculas
#   - Segundo nivel: la categoría asignada según el diccionario mapping
#
# Finalmente se calcula la media de cada grupo con .mean()

df2.groupby([str.lower, mapping]).mean()


### Ejemplo de *Grouping*

Como ejemplo de esto, en un par de líneas de código Python podemos juntar todo y contar los planetas descubiertos por método y por década:

In [ ]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

Esto demuestra el poder de la combinación de muchas de las operaciones que hemos visto hasta ahora cuando se observan conjuntos de datos más realistas.
Inmediatamente obtenemos una idea general de cuándo y cómo se han descubierto planetas en las últimas décadas.

Aquí sugeriría que profundicéis en estas pocas líneas de código, y evaluaseis los pasos individuales para aseguraros de que entendéis exactamente lo que están haciendo al resultado.
Es cierto que es ejemplo algo complicado, pero la comprensión de estas pequeñas píldoras os darán los medios para explorar de manera similar tus propios datos. :-)

# Pivot Tables

Hemos visto cómo la abstracción ``GroupBy`` nos permite explorar las relaciones dentro de un conjunto de datos.
Una *tabla pivotante* es una operación similar que suele verse en las hojas de cálculo y otros programas que operan con datos tabulares.
La tabla pivotante o *pivot table* toma como entrada datos simples en forma de columnas y agrupa las entradas en una tabla bidimensional que proporciona un resumen multidimensional de los datos.
La diferencia entre las *pivot tables* y ``GroupBy`` a veces puede causar confusión; **ayuda bastante pensar en las *pivot tables* como una versión *multidimensional* de la agregación ``GroupBy``.**
Es decir, divides-aplicas-combinas, pero tanto la división como la combinación no se producen en un índice unidimensional, **sino en una cuadrícula bidimensional.**


*Las tablas dinámicas en Excel (pivot tables) son herramientas para resumir y analizar grandes cantidades de datos de forma dinámica, permitiendo agrupar, filtrar y calcular valores rápidamente para identificar patrones y tendencias

## Motivación de las Pivot Tables

Para los ejemplos de esta sección, utilizaremos la base de datos de pasajeros del *Titanic*, disponible a través de la biblioteca **Seaborn**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
titanic = sns.load_dataset('titanic')

In [ ]:
#pivot 1

Contiene una gran cantidad de información sobre cada uno de los pasajeros de ese viaje algo maldito, incluyendo el género, la edad, la clase, la tarifa pagada y mucho más.

## Pivot Tables *a mano*

Para empezar a aprender más sobre estos datos, podríamos empezar por agrupar según el género, el estado de supervivencia o alguna combinación de ellos.
Como hemos comentado anteriormente, podrías verte tentado a aplicar una operación ``GroupBy``; por ejemplo, veamos la tasa de supervivencia por género:

In [ ]:
titanic.groupby('sex')[['survived']].mean()

Esto nos da inmediatamente una idea: en general, tres de cada cuatro mujeres a bordo sobrevivieron, mientras que sólo uno de cada cinco hombres lo hizo.

Esto es útil, pero podríamos ir un paso más allá y analizar la supervivencia por sexo y, por ejemplo, por clase. Utilizando el vocabulario de ``GroupBy``, podríamos proceder de la siguiente manera; agrupamos por clase y sexo, seleccionamos la supervivencia, aplicamos una media agregada, luego combinamos los grupos resultantes, y terminamos descomponemos el índice jerárquico para revelar la multidimensionalidad oculta. En código:

In [ ]:
#pivot 2

Esto nos da una mejor idea de cómo el género y la clase afectan a la supervivencia, pero el código empieza a parecer un poco confuso.
Aunque cada paso de esta cadena tiene sentido a la luz de las herramientas que hemos discutido previamente, la larga cadena de código no es particularmente fácil de leer o utilizar.
Este ``GroupBy`` bidimensional es lo suficientemente común como para que Pandas incluya una ruta más sencilla, ``pivot_table``, que maneja precisamente este tipo de agregación multidimensional.

## Sintaxis de las Pivot Table

Aquí está el equivalente a la operación anterior utilizando el método ``pivot_table`` de ``DataFrame``:

In [ ]:
titanic


In [ ]:
# Crear una tabla dinámica (pivot table) a partir del DataFrame 'titanic'
# 'survived' → es la columna cuyos valores queremos resumir
# index='sex' → las filas estarán definidas por el sexo (male, female)
# columns='class' → las columnas estarán definidas por la clase (First, Second, Third)
# aggfunc='mean' → calculamos la media de 'survived' para cada combinación
#   (como 'survived' es 0 o 1, la media representa la tasa de supervivencia)
titanic.pivot_table('survived', index='sex', columns='class', aggfunc='mean')

In [ ]:
#usamos .T para transponer
titanic.pivot_table('survived', index='class', columns='sex', aggfunc='mean')

#titanic.pivot_table('survived', index='class', columns='sex', aggfunc='mean').T


Esto es mucho más fácil de leer que el enfoque "por grupos", y produce el mismo resultado. Como cabría esperar de un crucero transatlántico de principios del siglo XX, el grado de supervivencia favorece tanto a las mujeres como a las clases superiores. Las mujeres de primera clase sobrevivieron con casi total seguridad (¡hola, Rose!), mientras que sólo uno de cada diez hombres de tercera clase sobrevivió (¡lo siento, Jack!).

### Pivot tables multi nivel

Al igual que en el ``GroupBy``, la agrupación en las tablas dinámicas puede especificarse con múltiples niveles, y a través de una serie de opciones.
Por ejemplo, podríamos estar interesados en ver la edad como una tercera dimensión.
Agruparemos la edad utilizando la función ``pd.cut``:

In [ ]:
titanic.head()

In [ ]:
# Divide la columna "age" del DataFrame titanic en intervalos definidos
pd.cut(titanic['age'], [0, 18, 80])


In [ ]:
# Clasifica las edades en dos grupos (0-18 → 'menores', 18-80 → 'mayores')
age = pd.cut(titanic['age'], [0, 18, 80], labels=['menores','mayores'])

# Crea una tabla dinámica: filas = sexo+grupo de edad, columnas = clase, valores = conteo de 'survived'
titanic.pivot_table('survived', index=['sex', age], columns='class', aggfunc='count')


También podemos aplicar la misma estrategia al trabajar con las columnas; vamos a añadir información sobre la tarifa pagada utilizando ``pd.qcut`` para calcular automáticamente los cuantiles:

In [ ]:
# Divide la columna 'fare' (tarifa) en 2 grupos de igual tamaño usando cuantiles
fare = pd.qcut(titanic['fare'], 2)

# Tabla dinámica: filas = sexo + grupo de edad, columnas = grupo de tarifa + clase, valores = conteo de 'survived'
titanic.pivot_table('survived', ['sex', age], [fare, 'class'])


El resultado es una agregación cuatridimensional con índices jerárquicos cosa que vimos en Pandas 2, mostrada en una cuadrícula que demuestra la relación entre los valores.

### Opciones de Pivot table adicionales:

La notación completa del método ``pivot_table`` de ``DataFrame`` es la siguiente:

```python
# Para Pandas 0.18
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```

Ya hemos visto ejemplos de los tres primeros argumentos; aquí echaremos un vistazo rápido a los restantes.
Dos de las opciones, ``fill_value`` y ``dropna``, tienen que ver con los datos que faltan y son bastante sencillas; no mostraremos ejemplos de ellas aquí.

La palabra clave ``aggfunc`` controla qué tipo de agregación se aplica, que es una media por defecto.
Al igual que en ``GroupBy``, la especificación de la agregación puede ser una cadena que represente una de las opciones más comunes (por ejemplo, ``sum``, ``mean``, ``count``, ``min``, ``max``, etc.) o una función que implemente una agregación (por ejemplo, ``np.sum()``, ``min()``, ``sum()``, etc.).
Además, puede especificarse como un diccionario que asigna una columna a cualquiera de las opciones deseadas anteriormente:

In [ ]:
# Tabla dinámica: filas = sexo, columnas = clase; y con aggfunc= muestra suma de sobrevivientes y tarifa media
titanic.pivot_table(index='sex', columns='class', aggfunc={'survived':sum, 'fare':'mean'})


Fíjate también en que hemos omitido la palabra clave ``values``; al especificar una asignación para ``aggfunc``, ésta se determina automáticamente.

A veces es útil calcular los totales a lo largo de cada agrupación.
Esto puede hacerse mediante la palabra clave ``margins``:

In [ ]:
# Tabla dinámica: filas = sexo, columnas = clase; calcula promedio de 'survived' y añade totales (margins=True)
titanic.pivot_table('survived', index='sex', columns='class', margins=True)


Aquí esto nos da automáticamente información sobre la tasa de supervivencia por género, la tasa de supervivencia por género y la tasa de supervivencia global del 38%, todo ello por cada categoría de clase.
La etiqueta de los márgenes puede especificarse con la palabra clave ``margins_name``, que por defecto es ``"All"``.

## Ejemplo: Datos de nacimiento

Como ejemplo más interesante, veamos los datos de libre acceso sobre los nacimientos en Estados Unidos, proporcionados por los Centros de Control de Enfermedades (CDC).
Estos datos pueden encontrarse en https://raw.githubusercontent.com/jakevdp/data-CDCbirths/master/births.csv
(este conjunto de datos ha sido analizado ampliamente por Andrew Gelman y su grupo; míra, por ejemplo, [esta entrada de blog](http://andrewgelman.com/2012/06/14/cool-ass-signal-processing-using-gaussian-processes/)):

In [ ]:
!curl -O https://raw.githubusercontent.com/jakevdp/data-CDCbirths/master/births.csv

In [ ]:
births = pd.read_csv('data/births.csv')

Si echamos un vistazo a los datos, vemos que son relativamente sencillos: contienen el número de nacimientos agrupados por fecha y sexo:

In [ ]:
births.head()

Podemos empezar a entender estos datos un poco más utilizando una tabla dinámica.
Añadamos una columna de década y veamos los nacimientos de hombres y mujeres en función de la década:

In [ ]:
births['decade'] = 10 * (births['year'] // 10)
births.pivot_table('births', index='decade', columns='gender', aggfunc='sum')

Vemos inmediatamente que los nacimientos masculinos superan a los femeninos en cada década.
Para ver esta tendencia un poco más claramente, podemos utilizar las herramientas de trazado incorporadas en Pandas para visualizar el número total de nacimientos por año:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
sns.set()  # use Seaborn styles
births.pivot_table('births', index='year', columns='gender', aggfunc='sum').plot()
plt.ylabel('total births per year');

Con una simple tabla dinámica y el método ``plot()``, podemos ver inmediatamente la tendencia anual de los nacimientos por género. A ojo, parece que en los últimos 50 años los nacimientos masculinos han superado a los femeninos en aproximadamente un 5%.